This exercise will require you to pull some data from the Qunadl API. Qaundl is currently the most widely used aggregator of financial market data.

As a first step, you will need to register a free account on the http://www.quandl.com website.

After you register, you will be provided with a unique API key, that you should store:

In [1]:
# Store the API key as a string - according to PEP8, constants are always named in all upper case
# Fetch from outside repo: 
with open('../../QuandlAPI', 'r') as f:
    API_KEY = f.read()

Qaundl has a large number of data sources, but, unfortunately, most of them require a Premium subscription. Still, there are also a good number of free datasets.

For this mini project, we will focus on equities data from the Frankfurt Stock Exhange (FSE), which is available for free. We'll try and analyze the stock prices of a company called Carl Zeiss Meditec, which manufactures tools for eye examinations, as well as medical lasers for laser eye surgery: https://www.zeiss.com/meditec/int/home.html. The company is listed under the stock ticker AFX_X.

You can find the detailed Quandl API instructions here: https://docs.quandl.com/docs/time-series

While there is a dedicated Python package for connecting to the Quandl API, we would prefer that you use the *requests* package, which can be easily downloaded using *pip* or *conda*. You can find the documentation for the package here: http://docs.python-requests.org/en/master/ 

Finally, apart from the *requests* package, you are encouraged to not use any third party Python packages, such as *pandas*, and instead focus on what's available in the Python Standard Library (the *collections* module might come in handy: https://pymotw.com/3/collections/ ).
Also, since you won't have access to DataFrames, you are encouraged to us Python's native data structures - preferably dictionaries, though some questions can also be answered using lists.
You can read more on these data structures here: https://docs.python.org/3/tutorial/datastructures.html

Keep in mind that the JSON responses you will be getting from the API map almost one-to-one to Python's dictionaries. Unfortunately, they can be very nested, so make sure you read up on indexing dictionaries in the documentation provided above.

In [2]:
# First, import the relevant modules.
# We are limiting ourselves to standard python libraries for this exercise
import requests # This is the only package from outside the standard library allowed in this project. 
import statistics # Lib/statistics.py is part of the Standard Library so we are allowed to use it here. 
import operator # Lib/operator.py is allowed. 

In [3]:
# build URLs for the quandl API, incorporating the times-series parameters and default values
def getURL(database='FRED', dataset='GDP', start='2000-01-01', end='2001-01-01', collapse='no', limit=0, order='asc', transform='no'): 
    URL = 'https://www.quandl.com/api/v3/datasets/%s/%s.json?api_key=%s' % (database, dataset, API_KEY)
    kwargs = {'limit':limit, 'start_date':start, 'end_date':end, 'order':order, 'collapse':collapse, 'transform':transform}
    for k, v in kwargs.items():
        URL += '&' + k + '=' + str(v)
    return URL

In [4]:
# Now, call the Quandl API and pull out a small sample of the data to get a glimpse
# into the JSON structure that will be returned

# All defaults for our test: 
testr = requests.get(getURL())
testr # show response

<Response [200]>

In [5]:
# Inspect the JSON structure of the object you created, and take note of how nested it is,
# as well as the overall structure
testr.json()

{'dataset': {'id': 120140,
  'dataset_code': 'GDP',
  'database_code': 'FRED',
  'name': 'Gross Domestic Product',
  'description': 'Units: Billions of Dollars\nSeasonal Adjustment: Seasonally Adjusted Annual Rate\nNotes: A Guide to the National Income and Product Accounts of the United States (NIPA) - (http://www.bea.gov/national/pdf/nipaguid.pdf)',
  'refreshed_at': '2021-03-26T01:13:27.583Z',
  'newest_available_date': '2020-10-01',
  'oldest_available_date': '1947-01-01',
  'column_names': ['Date', 'Value'],
  'frequency': 'quarterly',
  'type': 'Time Series',
  'premium': False,
  'limit': 0,
  'transform': None,
  'column_index': None,
  'start_date': '2000-01-01',
  'end_date': '2001-01-01',
  'data': [['2000-01-01', 10002.857],
   ['2000-04-01', 10247.679],
   ['2000-07-01', 10319.825],
   ['2000-10-01', 10439.025],
   ['2001-01-01', 10472.879]],
  'collapse': None,
  'order': 'asc',
  'database_id': 118}}

In [6]:
# We see a value would be located at dict['dataset']['data'][rownum][1]

These are your tasks for this mini project:

1. Collect data from the Frankfurt Stock Exchange, for the ticker AFX_X, for the whole year 2017 (keep in mind that the date format is YYYY-MM-DD).
2. Convert the returned JSON object into a Python dictionary.
3. Calculate what the highest and lowest opening prices were for the stock in this period.
4. What was the largest change in any one day (based on High and Low price)?
5. What was the largest change between any two days (based on Closing Price)?
6. What was the average daily trading volume during this year?
7. (Optional) What was the median trading volume during this year. (Note: you may need to implement your own function for calculating the median.)

In [10]:
URL = getURL('FSE', 'AFX_X', '2017-01-01', '2017-12-31')
# Query 
r = requests.get(URL)
r # Show response

<Response [200]>

The above HTTP code finally tells us **OK** so we will proceed. 

In [11]:
# Extract the nested data
columns = r.json()['dataset']['column_names']
data = r.json()['dataset']['data']

In [12]:
# We'll be collecting our calculations into a report
report = {}

The data is a list of rows with each row being a list of column values. 
So we will be able to access a single column using [row[colnum] for row in data]

In [13]:
columns

['Date',
 'Open',
 'High',
 'Low',
 'Close',
 'Change',
 'Traded Volume',
 'Turnover',
 'Last Price of the Day',
 'Daily Traded Units',
 'Daily Turnover']

In [14]:
# Find the index of the opening stock price column
open_id = columns.index('Open')
# Calculate what the highest and lowest opening prices were for the stock in this period.
report['highest_open'] = max([row[open_id] for row in data])
report['lowest_open'] = min([row[open_id] for row in data])

TypeError: '>' not supported between instances of 'NoneType' and 'float'

"NoneType" must mean there are some blank records so we will clean them up and try again. 

In [15]:
# Remove the bad records
cleandata = [row for row in data if type(row[open_id]) is not type(None)]

# Calculate what the highest and lowest opening prices were for the stock in this period.
report['Highest_Open'] = max([row[open_id] for row in cleandata])
report['Lowest_Open'] = min([row[open_id] for row in cleandata])

In [16]:
# What was the largest change in any one day (based on High and Low price)?
lows = [row[columns.index('Low')] for row in data] # Select the Low column
highs = [row[columns.index('High')] for row in data] # Select the High column
changes = list(map(operator.sub, highs, lows)) # 
report['Largest_Change'] = max(changes)

In [17]:
# What was the largest change between any two days (based on Closing Price)?

# Select the column of interest
closes = [row[columns.index('Close')] for row in data]
# Calculate the changes between daily closes
interdays = [x[0]-x[1] for x in zip(closes[1:], closes[0:-1])]
# Report the largest
report['Largest_Interday'] = max(interdays)

In [18]:
# What was the average daily trading volume during this year?

# Select the column of interest
volumes = [row[columns.index('Traded Volume')] for row in data]
# Report the average
report['Average_Traded_Volume'] = statistics.mean(volumes)

In [19]:
# What was the median trading volume during this year. 

# Report the median
report['Median_Traded_Volume'] = statistics.median(volumes)

For reference: 
- Calculate what the highest and lowest opening prices were for the stock in this period.
- What was the largest change in any one day (based on High and Low price)?
- What was the largest change between any two days (based on Closing Price)?
- What was the average daily trading volume during this year?
- What was the median trading volume during this year. 

In [20]:
# Display the report of our calculations
report

{'Highest_Open': 53.11,
 'Lowest_Open': 34.0,
 'Largest_Change': 2.8100000000000023,
 'Largest_Interday': 1.7199999999999989,
 'Average Traded Volume': 89124.33725490196,
 'Median Traded Volume': 76286.0}